In [1]:
import numpy as np
from scipy.constants import c, pi
import matplotlib.pyplot as plt
from scipy.special import jv, kv
import time
from scipy.optimize import brenth
import sys

In [2]:
class Eigensolver(object):
    def __init__(self,lmin, lmax, N):
        self.l_vec = np.linspace(lmin, lmax, N)
        
        self._A_ = {'sio2': [0.6965325, 0.0660932**2],
                    'ge': [0.7083925, 0.0853842**2]}
        self._B_ = {'sio2': [0.4083099, 0.1181101**2],
                    'ge': [0.4203993, 0.1024839**2]}
        self._C_ = {'sio2': [0.8968766, 9.896160**2],
                    'ge': [0.8663412, 9.896175**2]}
    
    def V_func(self,r):
        self.V_vec = r * (2 * pi / self.l_vec) * (self.ncore**2 - self.nclad**2)**0.5
        

    def indexes(self, l, r):
        per_core, per_clad = 'ge', 'sio2'

        self.A = [self._A_[str(per_core)], self._A_[str(per_clad)]]
        self.B = [self._B_[str(per_core)], self._B_[str(per_clad)]]
        self.C = [self._C_[str(per_core)], self._C_[str(per_clad)]]

        return self.sellmeier(l)
        
        
    def initialise_fibre(self, r):
        self.ncore, self.nclad = self.indexes(self.l_vec, r)
        self.r =r
        self.V_func(r)
        print('Maximum V number in space is {}'.format(np.max(self.V_vec)))
    
    def sellmeier(self, l):
        l = (l*1e6)**2
        n = []
        for a, b, c in zip(self.A, self.B, self.C):
            n.append(
                (1 + l*(a[0]/(l - a[1]) + b[0]/(l - b[1]) + c[0]/(l - c[1])))**0.5)
        return n    
    
    
    def equation_01(self, u, i):
        w = self.w_equation(u, i)
        return jv(0,u) / (u * jv(1,u)) - kv(0,w) / ( w * kv(1, w)) 
    
    
    def equation_11(self, u,i):
        w = self.w_equation(u, i)
        return jv(1,u) / (u * jv(0,u)) + kv(1,w) / ( w * kv(0, w))
    
    def w_equation(self, u, i):
        return (self.V_vec[i]**2 - u**2)**0.5
    
    
    def solve_01(self, i):
        return self.solve_equation(self.equation_01, i)
    
    def solve_11(self, i):
        return self.solve_equation(self.equation_11, i)
    
    
    def solve_equation(self, equation, i):
        margin = 1e-15
        m = margin
        #V_d = []
        s = []
        count = 8
        N_points = 2**count
        found_all = 2

        while found_all < 5:
            nm = len(s)
            u_vec = np.linspace(margin, self.V_vec[i] - margin, N_points)
            eq = equation(u_vec, i)
            s = np.where(np.sign(eq[:-1]) != np.sign(eq[1:]))[0] + 1
            count += 1
            N_points = 2**count
            if nm == len(s):
                found_all +=1
        u_sol, w_sol = np.zeros(len(s)), np.zeros(len(s))

        for iss, ss in enumerate(s):
            
            Rr = brenth(equation, u_vec[ss-1], u_vec[ss],
                        args=(i), full_output=True)
            #print(Rr)
            u_sol[iss] = Rr[0]
            w_sol[iss] = self.w_equation(Rr[0], i)

        if len(s) != 0:
            return u_sol, w_sol, self.neff(u_sol, w_sol, i)
        else:
            print(
                '----------------------------No solutions found for some inputs--------------------')
            print(' V = ', self.V_vec[i])
            print(' R = ', self.r)
            print(' l = ', self.l_vec[i])
            print(
                '----------------------------------------------------------------------------------')
            u = np.linspace(1e-6, self.V_vec[i] - 1e-6, 2048)

            e = equation(u, i)
            plt.plot(np.abs(u), e)

            plt.xlim(u.min(), u.max())
            plt.ylim(-10, 10)
            plt.show()
            sys.exit(1)
    def neff(self,u, w, i):
        return (((self.ncore[i]/ u)**2 + (self.nclad[i]/w)**2)/ (1/u**2 + 1/w**2))**0.5

In [16]:
class Modes(object):  
    def __init__(self, a, u, w, neff, lam_vec, grid_points, n = 1):
        self.N_points = grid_points
        self.set_coordinates(a)
        self.u = u
        self.w = w
        self.k = 2 *pi /lam_vec
        self.beta = neff * self.k
    
    def set_coordinates(self, a):
        self.x, self.y = np.linspace(-2*a, 2*a, self.N_points),\
                         np.linspace(-2*a, 2*a, self.N_points)
        self.X, self.Y = np.meshgrid(self.x,self.y)
        self.R = ((self.X)**2 + (self.Y)**2)**0.5
        self.T = np.arctan(self.Y/self.X)
        return None
    def E_r(self, r, theta):
        r0_ind = np.where(r <= self.a)
        r1_ind = np.where(r > self.a)
        temp = np.zeros(r.shape, dtype=np.complex128)
        r0, r1 = r[r0_ind], r[r1_ind]
        temp[r0_ind] = -1j * self.beta*self.a / \
            self.u*(0.5*(1 - self.s) * jv(self.n - 1, self.u * r0 / self.a)
                    - 0.5*(1 + self.s)*jv(self.n + 1, self.u * r0 / self.a))

        temp[r1_ind] = -1j * self.beta*self.a*jv(self.n, self.u)/(self.w*kv(self.n, self.w)) \
            * (0.5*(1 - self.s) * kv(self.n - 1, self.w * r1 / self.a)
               + 0.5*(1 + self.s)*kv(self.n+1, self.w * r1 / self.a))

        return temp*np.cos(self.n*theta), temp*np.cos(self.n*theta+pi/2)


    def E_theta(self, r, theta):
        r0_ind = np.where(r <= self.a)
        r1_ind = np.where(r > self.a)
        temp = np.zeros(r.shape, dtype=np.complex128)
        r0, r1 = r[r0_ind], r[r1_ind]
        temp[r0_ind] = 1j * self.beta*self.a / \
            self.u*(0.5*(1 - self.s) * jv(self.n - 1, self.u * r0 / self.a)
                    + 0.5*(1 + self.s)*jv(self.n+1, self.u * r0 / self.a))

        temp[r1_ind] = 1j * self.beta*self.a * \
            jv(self.n, self.u)/(self.w*kv(self.n, self.w)) \
            * (0.5*(1 - self.s) * kv(self.n - 1, self.w * r1 / self.a)
               - 0.5*(1 + self.s)*kv(self.n+1, self.w * r1 / self.a))
        return temp*np.sin(self.n*theta), temp*np.sin(self.n*theta+pi/2)


    def E_zeta(self, r, theta):
        r0_ind = np.where(r <= self.a)
        r1_ind = np.where(r > self.a)
        temp = np.zeros(r.shape, dtype=np.complex128)
        r0, r1 = r[r0_ind], r[r1_ind]
        temp[r0_ind] = jv(self.n, self.u*r0/self.a)
        temp[r1_ind] = jv(self.n, self.u) * \
            kv(self.n, self.w*r1/self.a)/kv(self.n, self.w)
        return temp*np.cos(self.n*theta), temp*np.cos(self.n*theta+pi/2)

    def E_carte(self):
        Er = self.E_r(self.R, self.T)
        Et = self.E_theta(self.R, self.T)
        Ex, Ey, Ez = [], [], []
        for i in range(len(Er)):
            Ex.append(Er[i] * np.cos(self.T) - Et[i] * np.sin(self.T))
            Ey.append(Er[i] * np.sin(self.T) + Et[i] * np.cos(self.T))
        Ez = self.E_zeta(self.R, self.T)

        return (Ex[0], Ey[0], Ez[0]), (Ex[1], Ey[1], Ez[1])

In [11]:
lmin = 1546e-9
lmax = 1555e-9
N = 10
a_vec = 5e-6

In [12]:
e = Eigensolver(lmin, lmax, N)

In [13]:
e.initialise_fibre(a_vec)

Maximum V number in space is 3.2170942149576436


In [14]:
u_01, w_01, neff01 = [np.zeros(N) for i in range(3)]
u_11, w_11, neff11 = [np.zeros([2,N]) for i in range(3)]

In [15]:
for i in range(N):
    u_01[i], w_01[i], neff01[i] = e.solve_01(i)
    u_11[:,i], w_11[:,i], neff11[:,i] = e.solve_11(i)